Phase 1 – Imports and Setup

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Phase 2 – Data Loading & Augmentation

In [ ]:
base_path = "C:\\Users\\LOQ\\OneDrive\\Documents\\Work Space\\PlantDocAI\\data"

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

valid_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(f"{base_path}\\train", transform=train_transforms)
valid_data = datasets.ImageFolder(f"{base_path}\\valid", transform=valid_transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False, num_workers=2)

print(f"Classes found: {train_data.classes}")
print(f"Train images: {len(train_data)}, Validation images: {len(valid_data)}")


Phase 3 – Enhanced Training: EarlyStopping & Fine-Tuning

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.amp import GradScaler
from torch.amp import autocast

In [ ]:
# 1. Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# 2. Paths & Transforms
base_path = r"C:\Users\LOQ\OneDrive\Documents\Work Space\PlantDocAI\data"

train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

valid_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_data = datasets.ImageFolder(f"{base_path}\\train", transform=train_transforms)
valid_data = datasets.ImageFolder(f"{base_path}\\valid", transform=valid_transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False, num_workers=2)

print(f"Classes: {train_data.classes[:10]} ... ({len(train_data.classes)} total)")

In [ ]:
# 3. Load ResNet50
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False

# Replace final classifier
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, len(train_data.classes)),
    nn.LogSoftmax(dim=1)
)
model = model.to(device)


In [ ]:
# 4. Loss, Optimizer, Scheduler, Mixed Precision
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
scaler = GradScaler(device="cuda") 

In [ ]:
# 5. EarlyStopping Setup
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_acc):
        if self.best_score is None:
            self.best_score = val_acc
        elif val_acc < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_acc
            self.counter = 0

early_stopping = EarlyStopping(patience=4)

In [ ]:
# 6. Training Loop
epochs = 15  
best_accuracy = 0.0
train_loss_list, train_acc_list, valid_loss_list, valid_acc_list = [], [], [], []

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    print("-"*30)

    # Training
    model.train()
    running_loss, running_correct = 0.0, 0
    for inputs, labels in tqdm(train_loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast(device_type="cuda"):  # specify device
         outputs = model(inputs)
         loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_correct += torch.sum(preds == labels.data)

    train_epoch_loss = running_loss / len(train_data)
    train_epoch_acc = running_correct.double() / len(train_data)
    train_loss_list.append(train_epoch_loss)
    train_acc_list.append(train_epoch_acc)

    # Validation
    model.eval()
    valid_loss, valid_correct = 0.0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc="Validating", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            valid_correct += torch.sum(preds == labels.data)

    valid_epoch_loss = valid_loss / len(valid_data)
    valid_epoch_acc = valid_correct.double() / len(valid_data)
    valid_loss_list.append(valid_epoch_loss)
    valid_acc_list.append(valid_epoch_acc)

    print(f"Train Loss: {train_epoch_loss:.4f} | Train Acc: {train_epoch_acc:.4f} | "
          f"Valid Loss: {valid_epoch_loss:.4f} | Valid Acc: {valid_epoch_acc:.4f}")

    # Step the scheduler
    scheduler.step()

    # Save best model
    if valid_epoch_acc > best_accuracy:
        best_accuracy = valid_epoch_acc
        torch.save(model.state_dict(), "best_resnet50_model.pth")
        print("Best model saved!")

    # EarlyStopping check
    early_stopping(valid_epoch_acc)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Gradual Unfreezing after few epochs
    if epoch == 5: 
        print("Unfreezing last 2 layers for fine-tuning")
        for name, param in model.named_parameters():
            if "layer4" in name:
                param.requires_grad = True
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
        scheduler = StepLR(optimizer, step_size=3, gamma=0.1)

print(f"\nTraining Complete! Best Validation Accuracy: {best_accuracy:.4f}")

In [ ]:
# 7. Plot Metrics
train_acc_cpu = [acc.cpu().numpy() if torch.is_tensor(acc) else acc for acc in train_acc_list]
valid_acc_cpu = [acc.cpu().numpy() if torch.is_tensor(acc) else acc for acc in valid_acc_list]

plt.figure(figsize=(12,5))

# Loss Curve
plt.subplot(1,2,1)
plt.plot(train_loss_list, label='Train Loss')
plt.plot(valid_loss_list, label='Valid Loss')
plt.legend()
plt.title('Loss Curve')

# Accuracy Curve
plt.subplot(1,2,2)
plt.plot(train_acc_cpu, label='Train Acc')
plt.plot(valid_acc_cpu, label='Valid Acc')
plt.legend()
plt.title('Accuracy Curve')

plt.show()


Phase 4 – Plant Disease Prediction

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import json

# 1. Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Load the trained model
num_classes = 38  
model = models.resnet50(weights=None) 
num_features = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Linear(num_features, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(256, num_classes),
    torch.nn.LogSoftmax(dim=1)
)
model.load_state_dict(torch.load("best_resnet50_model.pth"))
model = model.to(device)
model.eval()
print("✅ Model loaded for inference")

# 3. Image Transform
predict_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# ===== 4. Load Class Names =====
classes = [
    'Apple___Apple_scab',
    'Apple___Black_rot',
    'Apple___Cedar_apple_rust',
    'Apple___healthy',
    'Blueberry___healthy',
    'Cherry_(including_sour)___Powdery_mildew',
    'Cherry_(including_sour)___healthy',
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
    'Corn_(maize)___Common_rust_',
    'Corn_(maize)___Northern_Leaf_Blight',
    'Corn_(maize)___healthy',
    'Grape___Black_rot',
    'Grape___Esca_(Black_Measles)',
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
    'Grape___healthy',
    'Orange___Haunglongbing_(Citrus_greening)',
    'Peach___Bacterial_spot',
    'Peach___healthy',
    'Pepper,_bell___Bacterial_spot',
    'Pepper,_bell___healthy',
    'Potato___Early_blight',
    'Potato___Late_blight',
    'Potato___healthy',
    'Raspberry___healthy',
    'Soybean___healthy',
    'Squash___Powdery_mildew',
    'Strawberry___Leaf_scorch',
    'Strawberry___healthy',
    'Tomato___Bacterial_spot',
    'Tomato___Early_blight',
    'Tomato___Late_blight',
    'Tomato___Leaf_Mold',
    'Tomato___Septoria_leaf_spot',
    'Tomato___Spider_mites Two-spotted_spider_mite',
    'Tomato___Target_Spot',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
    'Tomato___Tomato_mosaic_virus',
    'Tomato___healthy'
]


# ===== 5. Disease Treatment Dictionary =====
treatment_dict = {
    # ===== APPLE =====
    "Apple___Apple_scab": """
Cause: Fungus *Venturia inaequalis*.
Symptoms: Olive-green spots on leaves and fruits; cracking on apple skin.
Treatment: Use fungicides like Mancozeb or Captan every 10–14 days.
Products: Dithane M-45 (Mancozeb), Captaf, or Score (Difenoconazole).
Prevention: Prune infected twigs, remove fallen leaves, ensure good air circulation.
""",

    "Apple___Black_rot": """
Cause: Fungus *Botryosphaeria obtusa*.
Symptoms: Dark concentric rings on fruit, leaf spots, twig cankers.
Treatment: Apply Captan or Thiophanate-methyl during early season.
Products: Topsin-M, Captaf, Mancozeb sprays.
Prevention: Remove mummified fruits, prune infected branches, avoid overhead irrigation.
""",

    "Apple___Cedar_apple_rust": """
Cause: Fungus *Gymnosporangium juniperi-virginianae*.
Symptoms: Orange-yellow leaf spots with fringes; fruit deformation.
Treatment: Use fungicides like Myclobutanil or Mancozeb.
Products: Rally 40WSP (Myclobutanil), Dithane M-45.
Prevention: Remove nearby cedar trees, apply fungicide preventively in spring.
""",

    "Apple___healthy": "No disease detected. Keep monitoring and follow preventive care (pruning, sanitation, regular spraying).",

    # ===== BLUEBERRY =====
    "Blueberry___healthy": "Healthy plant. Maintain pH 4.5–5.5, ensure proper irrigation and mulching.",

    # ===== CHERRY =====
    "Cherry_(including_sour)___Powdery_mildew": """
Cause: Fungus *Podosphaera clandestina*.
Symptoms: White powdery coating on young leaves and fruits.
Treatment: Use Sulfur-based fungicides or Myclobutanil.
Products: Wettable sulfur, Bayleton.
Prevention: Prune infected shoots, ensure good airflow.
""",

    "Cherry_(including_sour)___healthy": "Healthy cherry. Maintain pruning and avoid overhead watering.",

    # ===== CORN (MAIZE) =====
    "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot": """
Cause: Fungus *Cercospora zeae-maydis*.
Symptoms: Long gray lesions on leaves reducing photosynthesis.
Treatment: Apply Strobilurin or Triazole fungicides.
Products: Azoxystrobin (Amistar), Propiconazole (Tilt).
Prevention: Rotate crops, use resistant hybrids, avoid dense planting.
""",

    "Corn_(maize)___Common_rust_": """
Cause: Fungus *Puccinia sorghi*.
Symptoms: Reddish-brown pustules on upper and lower leaf surfaces.
Treatment: Apply fungicides like Azoxystrobin or Mancozeb.
Products: Amistar, Dithane M-45.
Prevention: Plant resistant varieties, rotate crops.
""",

    "Corn_(maize)___Northern_Leaf_Blight": """
Cause: Fungus *Exserohilum turcicum*.
Symptoms: Large cigar-shaped gray-green lesions on leaves.
Treatment: Use Propiconazole or Mancozeb.
Products: Tilt, Dithane M-45.
Prevention: Rotate crops, plow residue, plant resistant hybrids.
""",

    "Corn_(maize)___healthy": "Healthy maize plant. Maintain nitrogen balance and weed control.",

    # ===== GRAPE =====
    "Grape___Black_rot": """
Cause: Fungus *Guignardia bidwellii*.
Symptoms: Circular brown lesions with black margins on leaves and fruits.
Treatment: Apply Captan, Mancozeb, or Myclobutanil.
Products: Dithane, Rally, Captaf.
Prevention: Remove mummified berries, prune infected vines.
""",

    "Grape___Esca_(Black_Measles)": """
Cause: Fungus complex including *Phaeomoniella chlamydospora*.
Symptoms: Tiger-stripe leaf patterns, fruit shrivel, internal wood rot.
Treatment: Remove infected vines, apply trunk protectants (Trichoderma-based biocontrol).
Products: BioTricho, Serenade (Bacillus subtilis).
Prevention: Avoid pruning wounds in wet conditions.
""",

    "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)": """
Cause: Fungus *Isariopsis griseola*.
Symptoms: Angular dark brown spots on leaves, leading to defoliation.
Treatment: Spray Mancozeb or Copper oxychloride.
Products: Dithane M-45, Blue Copper.
Prevention: Remove infected debris, ensure ventilation.
""",

    "Grape___healthy": "Healthy vine. Continue canopy management and balanced nutrition.",

    # ===== ORANGE =====
    "Orange___Haunglongbing_(Citrus_greening)": """
Cause: Bacterium *Candidatus Liberibacter asiaticus* transmitted by psyllid insects.
Symptoms: Yellow shoots, lopsided fruits, bitter taste.
Treatment: No cure; control psyllids using Imidacloprid.
Products: Confidor (Imidacloprid), Neem oil spray.
Prevention: Remove infected trees, replant resistant varieties.
""",

    # ===== PEACH =====
    "Peach___Bacterial_spot": """
Cause: *Xanthomonas campestris pv. pruni*.
Symptoms: Water-soaked spots on leaves and fruits, premature drop.
Treatment: Copper-based bactericides.
Products: Copper oxychloride, Kocide 2000.
Prevention: Avoid overhead watering, prune infected shoots.
""",

    "Peach___healthy": "Healthy peach plant. Maintain pruning and copper spray schedule.",

    # ===== PEPPER (BELL) =====
    "Pepper,_bell___Bacterial_spot": """
Cause: *Xanthomonas campestris pv. vesicatoria*.
Symptoms: Small dark spots with yellow halos on leaves and fruits.
Treatment: Use Copper-based sprays and Streptomycin (seed treatment).
Products: Kocide, Streptocycline, Neem oil.
Prevention: Avoid overhead irrigation, disinfect tools.
""",

    "Pepper,_bell___healthy": "Healthy pepper. Monitor for aphids and maintain proper spacing.",

    # ===== POTATO =====
    "Potato___Early_blight": """
Cause: Fungus *Alternaria solani*.
Symptoms: Dark concentric leaf spots, early leaf death.
Treatment: Mancozeb, Chlorothalonil, or Azoxystrobin.
Products: Dithane, Bravo, Amistar.
Prevention: Rotate crops, remove debris, avoid nitrogen excess.
""",

    "Potato___Late_blight": """
Cause: *Phytophthora infestans*.
Symptoms: Water-soaked dark lesions on leaves and stems.
Treatment: Use Metalaxyl + Mancozeb combination.
Products: Ridomil Gold, Curzate M8.
Prevention: Avoid overhead irrigation, destroy infected tubers.
""",

    "Potato___healthy": "Healthy potato crop. Maintain soil moisture and spacing.",

    # ===== RASPBERRY =====
    "Raspberry___healthy": "Healthy raspberry. Maintain cane pruning and pest monitoring.",

    # ===== SOYBEAN =====
    "Soybean___healthy": "Healthy soybean. Rotate crops and control weeds to prevent rust.",

    # ===== SQUASH =====
    "Squash___Powdery_mildew": """
Cause: Fungus *Erysiphe cichoracearum*.
Symptoms: White powdery growth on leaves, reduced vigor.
Treatment: Apply Sulfur or Potassium bicarbonate sprays.
Products: Wettable sulfur, Milstop.
Prevention: Provide airflow, avoid overhead watering.
""",

    # ===== STRAWBERRY =====
    "Strawberry___Leaf_scorch": """
Cause: Fungus *Diplocarpon earlianum*.
Symptoms: Purple spots that merge and scorch leaves.
Treatment: Captan or Chlorothalonil sprays.
Products: Captaf, Bravo.
Prevention: Use drip irrigation, remove infected leaves.
""",

    "Strawberry___healthy": "Healthy strawberry. Maintain mulching and pest monitoring.",

    # ===== TOMATO =====
    "Tomato___Bacterial_spot": """
Cause: *Xanthomonas campestris pv. vesicatoria*.
Symptoms: Greasy dark spots on leaves and fruit.
Treatment: Copper oxychloride or Streptomycin sprays.
Products: Kocide, Streptocycline.
Prevention: Avoid overhead watering, disinfect seeds.
""",

    "Tomato___Early_blight": """
Cause: *Alternaria solani*.
Symptoms: Brown concentric leaf spots, leaf yellowing.
Treatment: Mancozeb, Chlorothalonil, or Azoxystrobin.
Products: Dithane, Bravo, Amistar.
Prevention: Rotate crops, prune lower leaves.
""",

    "Tomato___Late_blight": """
Cause: *Phytophthora infestans*.
Symptoms: Water-soaked lesions on leaves and fruits, white fungal growth.
Treatment: Metalaxyl + Mancozeb.
Products: Ridomil Gold, Curzate.
Prevention: Avoid excess humidity, destroy infected plants.
""",

    "Tomato___Leaf_Mold": """
Cause: Fungus *Passalora fulva*.
Symptoms: Yellow patches on upper leaf surface, gray mold below.
Treatment: Use Copper fungicides or Chlorothalonil.
Products: Kocide, Bravo.
Prevention: Ensure ventilation, control humidity.
""",

    "Tomato___Septoria_leaf_spot": """
Cause: Fungus *Septoria lycopersici*.
Symptoms: Small dark spots with gray centers on older leaves.
Treatment: Mancozeb or Chlorothalonil.
Products: Dithane, Bravo.
Prevention: Remove lower leaves, avoid splashing water.
""",

    "Tomato___Spider_mites Two-spotted_spider_mite": """
Cause: Pest *Tetranychus urticae*.
Symptoms: Fine yellow spots, webbing on leaves.
Treatment: Use Abamectin or Neem oil.
Products: Vertimec (Abamectin), Neemazal.
Prevention: Maintain humidity, spray miticide at threshold.
""",

    "Tomato___Target_Spot": """
Cause: *Corynespora cassiicola*.
Symptoms: Concentric spots on leaves and fruits.
Treatment: Mancozeb or Azoxystrobin.
Products: Dithane, Amistar.
Prevention: Avoid dense planting, rotate crops.
""",

    "Tomato___Tomato_Yellow_Leaf_Curl_Virus": """
Cause: Whitefly-transmitted Begomovirus.
Symptoms: Upward leaf curling, stunted growth, yellowing.
Treatment: Control whiteflies with Imidacloprid or Neem oil.
Products: Confidor, Neemazal.
Prevention: Remove infected plants, use resistant varieties.
""",

    "Tomato___Tomato_mosaic_virus": """
Cause: *Tobamovirus*.
Symptoms: Mottled, curled leaves; poor fruiting.
Treatment: No cure; remove infected plants.
Products: Disinfect tools with bleach.
Prevention: Use certified seeds, sanitize hands and tools.
""",

    "Tomato___healthy": "Healthy tomato. Maintain crop hygiene and monitor humidity."
}

# ===== 6. Structured Prediction Function =====
import torch
from PIL import Image
import matplotlib.pyplot as plt

def predict_disease_structured(image_path):
    """
    Predicts plant disease from an image and returns structured treatment info.
    
    Returns:
        dict: {
            'class_name': str,
            'cause': str,
            'symptoms': str,
            'treatment': str,
            'products': str,
            'prevention': str
        }
    """
    try:
        # Load and preprocess image
        img = Image.open(image_path).convert("RGB")
        img_tensor = predict_transform(img).unsqueeze(0).to(device)

        # Model inference
        model.eval()
        with torch.no_grad():
            outputs = model(img_tensor)
            _, pred = torch.max(outputs, 1)

        # Get class name
        class_name = classes[pred.item()]

        # Get detailed treatment info
        raw_info = treatment_dict.get(class_name, "No treatment info available.")

        # Parse structured info (optional: split by lines)
        info_lines = raw_info.strip().split("\n")
        structured_info = {
            "class_name": class_name,
            "cause": "",
            "symptoms": "",
            "treatment": "",
            "products": "",
            "prevention": ""
        }

        for line in info_lines:
            if line.startswith("Cause:"):
                structured_info["cause"] = line.replace("Cause:", "").strip()
            elif line.startswith("Symptoms:"):
                structured_info["symptoms"] = line.replace("Symptoms:", "").strip()
            elif line.startswith("Treatment:"):
                structured_info["treatment"] = line.replace("Treatment:", "").strip()
            elif line.startswith("Products:"):
                structured_info["products"] = line.replace("Products:", "").strip()
            elif line.startswith("Prevention:"):
                structured_info["prevention"] = line.replace("Prevention:", "").strip()

        # Display image with predicted class
        plt.imshow(img)
        plt.title(f"Prediction: {class_name}")
        plt.axis("off")
        plt.show()

        # Print structured info in console
        print(f"🪴 Prediction: {structured_info['class_name']}")
        print("💊 Recommended Treatment & Details:")
        print(f"Cause: {structured_info['cause']}")
        print(f"Symptoms: {structured_info['symptoms']}")
        print(f"Treatment: {structured_info['treatment']}")
        print(f"Products: {structured_info['products']}")
        print(f"Prevention: {structured_info['prevention']}")

        return structured_info

    except Exception as e:
        print(f"⚠️ Error during prediction: {e}")
        return None


# ===== 7. Test Prediction =====
test_image = r"C:\Users\LOQ\Downloads\potato.jpeg"
result = predict_disease_structured(test_image)

if result:
    print("\n✅ Structured Output Ready for App/GUI:")
    print(result)



In [ ]:
ph

In [ ]:
import os

def predict_disease_batch(folder_path):
    """
    Predicts plant disease for all images in a folder.
    
    Args:
        folder_path (str): Path to folder containing images.
        
    Returns:
        list of dicts: Each dict contains structured prediction for an image.
    """
    results = []
    supported_formats = (".jpg", ".jpeg", ".png", ".bmp")

    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(supported_formats):
            image_path = os.path.join(folder_path, filename)
            print(f"\nProcessing: {filename}")
            
            # Use the structured prediction function
            result = predict_disease_structured(image_path)
            
            if result:
                result["image_name"] = filename
                results.append(result)
            else:
                print(f"⚠️ Failed to predict for {filename}")

    print(f"\n✅ Batch prediction completed. Total images processed: {len(results)}")
    return results


In [ ]:
# Path to your test folder
test_folder = r"C:\Users\LOQ\OneDrive\Documents\Work Space\PlantDocAI\data\test"

# Run batch predictions
batch_results = predict_disease_batch(test_folder)

# Print all results
for res in batch_results:
    print(res)


In [ ]:
# app.py
import streamlit as st
from PIL import Image
import torch
from torchvision import transforms, models
from treatment_dict import treatment_dict  # Your detailed dictionary
import os
import pandas as pd
import datetime

# ------------------------
# App Config
# ------------------------
st.set_page_config(
    page_title="Plant Disease Detector",
    page_icon="🌿",
    layout="wide"
)

st.title("🌿 Plant Disease Detection & Treatment Recommendation")
st.markdown("Upload a plant image or a folder of images to get instant disease predictions and treatment recommendations.")

# ------------------------
# Device & Model Setup
# ------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 38
classes = [
    'Apple___Apple_scab','Apple___Black_rot','Apple___Cedar_apple_rust','Apple___healthy',
    'Blueberry___healthy','Cherry_(including_sour)___Powdery_mildew','Cherry_(including_sour)___healthy',
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot','Corn_(maize)___Common_rust_',
    'Corn_(maize)___Northern_Leaf_Blight','Corn_(maize)___healthy','Grape___Black_rot',
    'Grape___Esca_(Black_Measles)','Grape___Leaf_blight_(Isariopsis_Leaf_Spot)','Grape___healthy',
    'Orange___Haunglongbing_(Citrus_greening)','Peach___Bacterial_spot','Peach___healthy',
    'Pepper,_bell___Bacterial_spot','Pepper,_bell___healthy','Potato___Early_blight',
    'Potato___Late_blight','Potato___healthy','Raspberry___healthy','Soybean___healthy',
    'Squash___Powdery_mildew','Strawberry___Leaf_scorch','Strawberry___healthy','Tomato___Bacterial_spot',
    'Tomato___Early_blight','Tomato___Late_blight','Tomato___Leaf_Mold','Tomato___Septoria_leaf_spot',
    'Tomato___Spider_mites Two-spotted_spider_mite','Tomato___Target_Spot',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus','Tomato___Tomato_mosaic_virus','Tomato___healthy'
]

# Load model
model = models.resnet50(weights=None)
num_features = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Linear(num_features, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(256, num_classes),
    torch.nn.LogSoftmax(dim=1)
)
model.load_state_dict(torch.load("best_resnet50_model.pth", map_location=device))
model = model.to(device)
model.eval()

predict_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# ------------------------
# Prediction Function
# ------------------------
def predict_disease(image):
    img = image.convert("RGB")
    img_tensor = predict_transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(img_tensor)
        _, pred = torch.max(outputs, 1)
    class_name = classes[pred.item()]
    info = treatment_dict.get(class_name, "No treatment info available.")
    return class_name, info

# ------------------------
# Convert results to CSV
# ------------------------
def convert_results_to_csv(results):
    df = pd.DataFrame(results)
    return df.to_csv(index=False).encode('utf-8')

# ------------------------
# Tabs: Single vs Batch
# ------------------------
tab1, tab2 = st.tabs(["Single Image Prediction", "Batch Prediction"])
all_results = []

# ------------------------
# Single Image Prediction
# ------------------------
with tab1:
    uploaded_file = st.file_uploader("Upload a single image", type=["jpg","jpeg","png"])
    if uploaded_file:
        image = Image.open(uploaded_file)
        col1, col2 = st.columns([1,2])
        with col1:
            st.image(image, caption=uploaded_file.name, use_column_width=True)
        with col2:
            class_name, info = predict_disease(image)
            
            structured_info = {}
            for line in info.strip().split("\n"):
                if ":" in line:
                    key, value = line.split(":",1)
                    structured_info[key.strip()] = value.strip()
                else:
                    structured_info[line.strip()] = ""
            
            # Color coding
            color = "green" if "healthy" in class_name.lower() else "red"
            st.markdown(f"<div style='border-left:5px solid {color}; padding:10px; border-radius:5px;'>", unsafe_allow_html=True)
            st.subheader(f"🪴 Predicted Disease: {class_name}")
            for key, value in structured_info.items():
                with st.expander(key):
                    st.write(value)
            st.markdown("</div>", unsafe_allow_html=True)

            # Store result
            result_dict = {"image_name": uploaded_file.name, "class_name": class_name}
            result_dict.update(structured_info)
            all_results.append(result_dict)

# ------------------------
# Batch Prediction
# ------------------------
with tab2:
    folder_path = st.text_input("Enter folder path for images")
    if folder_path and os.path.exists(folder_path):
        images = [f for f in os.listdir(folder_path) if f.lower().endswith((".jpg",".jpeg",".png"))]
        st.write(f"Found {len(images)} images.")
        
        cols = st.columns(3)
        for idx, img_name in enumerate(images):
            image = Image.open(os.path.join(folder_path, img_name))
            col = cols[idx % 3]
            col.image(image, caption=img_name, width=200)
            
            class_name, info = predict_disease(image)
            
            structured_info = {}
            for line in info.strip().split("\n"):
                if ":" in line:
                    key, value = line.split(":",1)
                    structured_info[key.strip()] = value.strip()
                else:
                    structured_info[line.strip()] = ""
            
            color = "green" if "healthy" in class_name.lower() else "red"
            st.markdown(f"<div style='border-left:5px solid {color}; padding:10px; border-radius:5px;'>", unsafe_allow_html=True)
            st.subheader(f"🪴 Predicted Disease: {class_name}")
            for key, value in structured_info.items():
                with st.expander(key):
                    st.write(value)
            st.markdown("</div>", unsafe_allow_html=True)
            
            result_dict = {"image_name": img_name, "class_name": class_name}
            result_dict.update(structured_info)
            all_results.append(result_dict)

# ------------------------
# Download Results
# ------------------------
if all_results:
    st.markdown("---")
    st.markdown("### 📥 Download All Predictions")
    csv = convert_results_to_csv(all_results)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    st.download_button(
        label="Download CSV",
        data=csv,
        file_name=f"plant_disease_predictions_{timestamp}.csv",
        mime="text/csv"
    )


In [51]:
!pip uninstall fpdf -y
!pip uninstall fpdf2 -y
!pip install fpdf2


Found existing installation: fpdf2 2.8.4
Uninstalling fpdf2-2.8.4:
  Successfully uninstalled fpdf2-2.8.4
  Using cached fpdf2-2.8.4-py2.py3-none-any.whl.metadata (72 kB)
Using cached fpdf2-2.8.4-py2.py3-none-any.whl (251 kB)


In [52]:
import fpdf
print(fpdf.__version__)


AttributeError: module 'fpdf' has no attribute '__version__'